# Basic Chatbot

In [46]:
# Store Access Token as an environment variable
import getpass
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [47]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 24, 'total_tokens': 28, 'completion_time': 0.003333333, 'prompt_time': 0.004506276, 'queue_time': 0.166631092, 'total_time': 0.007839609}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--4e0f0d25-4d22-4e98-84f6-2e2addb2411e-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28})

# Search Engine

## Loading up search engine

We will be using Tavily (a search engine) as a tool. In order to use it, you will need to get and set an API key:
https://tavily.com/

In [18]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [24]:
# Loading up search engine 
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in hk")
print(search_results)

[{'title': 'Weather in Hong Kong', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Hong Kong', 'region': '', 'country': 'Hong Kong', 'lat': 22.283, 'lon': 114.15, 'tz_id': 'Asia/Hong_Kong', 'localtime_epoch': 1747043726, 'localtime': '2025-05-12 17:55'}, 'current': {'last_updated_epoch': 1747043100, 'last_updated': '2025-05-12 17:45', 'temp_c': 27.2, 'temp_f': 81.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 12.3, 'wind_kph': 19.8, 'wind_degree': 120, 'wind_dir': 'ESE', 'pressure_mb': 1008.0, 'pressure_in': 29.77, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 48, 'cloud': 0, 'feelslike_c': 29.0, 'feelslike_f': 84.2, 'windchill_c': 24.7, 'windchill_f': 76.4, 'heatindex_c': 26.1, 'heatindex_f': 79.0, 'dewpoint_c': 17.6, 'dewpoint_f': 63.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.3, 'gust_mph': 16.6, 'gust_kph': 26.7}}", 'score': 0.9876443}, {'title': 'Weather in Hong Kong in

## Combine search engine with LLM by agent

In [26]:
tools = [search]
model_with_tools = model.bind_tools(tools)

In [28]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF? You should use web search.")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': "What's the weather in San Francisco?"}, 'id': 'call_y72h', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

In [29]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [32]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the weather in SF?")]}
)
response["messages"]

[HumanMessage(content="What's the weather in SF?", additional_kwargs={}, response_metadata={}, id='a297c731-228b-4861-b79c-349f4a9b64f3'),
 AIMessage(content='The current weather in San Francisco is sunny with a high of 68°F (20°C) and a low of 56°F (13°C).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 1912, 'total_tokens': 1943, 'completion_time': 0.025833333, 'prompt_time': 0.237873462, 'queue_time': -0.294430081, 'total_time': 0.263706795}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--9c543b6e-402f-4537-99c5-85fd84227f4f-0', usage_metadata={'input_tokens': 1912, 'output_tokens': 31, 'total_tokens': 1943})]

The above response indicate the model do not use tavily. So, we should add SystemMessage to instruct the agent to use web search if needed

In [39]:
# Adding SystemMessage to instruct the agent to use web search if needed
response = agent_executor.invoke(
    {"messages": [SystemMessage(content='Whenever you do not know the answer or need up-to-date information, perform a web search to find accurate and relevant results before responding.'), 
                  HumanMessage(content="What's the weather in SF? ")]}
)
response["messages"]

[SystemMessage(content='Whenever you do not know the answer or need up-to-date information, perform a web search to find accurate and relevant results before responding.', additional_kwargs={}, response_metadata={}, id='6ea97751-6268-4b30-9866-b5328952a886'),
 HumanMessage(content="What's the weather in SF? ", additional_kwargs={}, response_metadata={}, id='0b671691-727e-4040-86d3-d2bf8f52e683'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xpz7', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 1975, 'total_tokens': 2055, 'completion_time': 0.066666667, 'prompt_time': 0.246168528, 'queue_time': -0.302638948, 'total_time': 0.312835195}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9eda1e98-4db8-4a97-98df-f1ff51c1274f-0', too

## Adding in memory

The agent is stateless without MemorySaver. To make it knows which thread/conversation to resume from, we also have to pass in a thread_id when invoking the agent. 
Use LangGraph SqliteSaver or PostgresSaver and connect a database for persistent store. 

In [40]:
from langgraph.checkpoint.memory import MemorySaver

# In-Memory Saver, for demo only
memory = MemorySaver()

In [41]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Change thread_id for different conversation
config = {"configurable": {"thread_id": "abc123"}}

In [42]:
# 1st conversation
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 1906, 'total_tokens': 1910, 'completion_time': 0.003333333, 'prompt_time': 0.237213137, 'queue_time': -0.289303203, 'total_time': 0.24054647}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run--86f814cc-e641-44a9-81ef-828fc8c82ab5-0', usage_metadata={'input_tokens': 1906, 'output_tokens': 4, 'total_tokens': 1910})]}}
----


In [43]:
# 2nd conversation
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 961, 'total_tokens': 967, 'completion_time': 0.005, 'prompt_time': 0.119936318, 'queue_time': 0.166901968, 'total_time': 0.124936318}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--6cea7bf1-1f1f-42f5-88b7-c3f43cec57b5-0', usage_metadata={'input_tokens': 961, 'output_tokens': 6, 'total_tokens': 967})]}}
----


The agent replied "Your name is Bob!" which means it remember the conversation. 

### PostgreSQL as Memory Store

In [29]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the database URL
DATABASE_URL = os.getenv('DATABASE_URL')

In [48]:
from langchain_core.runnables import RunnableConfig
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.store.postgres import PostgresStore
from langgraph.store.base import BaseStore
import uuid

with (
    PostgresStore.from_conn_string(DATABASE_URL) as store,
    PostgresSaver.from_conn_string(DATABASE_URL) as checkpointer,
):
    #store.setup()
    #checkpointer.setup()

    def call_model(
        state: MessagesState,
        config: RunnableConfig,
        *,
        store: BaseStore,
    ):
        user_id = config["configurable"]["user_id"]
        namespace = ("memories", user_id)
        memories = store.search(namespace, query=str(state["messages"][-1].content))
        info = "\n".join([d.value["data"] for d in memories])
        system_msg = f"You are a helpful assistant talking to the user. User info: {info}"

        # Store new memories if the user asks the model to remember
        last_message = state["messages"][-1]
        if "remember" in last_message.content.lower():
            memory = "User name is Bob"
            store.put(namespace, str(uuid.uuid4()), {"data": memory})

        response = model.invoke(
            [{"role": "system", "content": system_msg}] + state["messages"]
        )
        return {"messages": response}

    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_edge(START, "call_model")

    graph = builder.compile(
        checkpointer=checkpointer,
        store=store,
    )

    config = {
        "configurable": {
            "thread_id": "1",
            "user_id": "1",
        }
    }
    for chunk in graph.stream(
        {"messages": [{"role": "user", "content": "Hi! Remember: my name is Bob"}]},
        config,
        stream_mode="values",
    ):
        chunk["messages"][-1].pretty_print()

    config = {
        "configurable": {
            "thread_id": "2",
            "user_id": "1",
        }
    }

    for chunk in graph.stream(
        {"messages": [{"role": "user", "content": "what is my name?"}]},
        config,
        stream_mode="values",
    ):
        chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! Remember: my name is Bob
================================== Ai Message ==================================

Hello Bob! I remember your name is Bob.
================================ Human Message =================================

what is my name?
================================== Ai Message ==================================

Your name is Bob! Nice to meet you, Bob!
